### Possibly interesting features based on https://news.ycombinator.com/item?id=36590226
- Time of day [Done]
- how many posts 1 hr before post made (indicative of whether it's peak hour or not)
- Analysis on title: TF-IDF? Remove stopwords? Any other importance measures, and onehot encode the impt words?

In [52]:
import os
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,sum,desc,when,udf

In [23]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [3]:
# Change file path
os.chdir("/Users/hydraze/Library/CloudStorage/GoogleDrive-tohziyu2@gmail.com/My Drive/Studies/KU Leuven/Courses/Classes/Y1S2/Advanced Analytics in Business/Project/3/AdvancedAnalytics_Streaming-Text-Analytics")
os.listdir()

['.DS_Store',
 'README.md',
 '.gitattributes',
 '.ipynb_checkpoints',
 '.git',
 'data']

In [205]:
# Load all folder names, then read into one textfile
file_path = "/Users/hydraze/Library/CloudStorage/GoogleDrive-tohziyu2@gmail.com/My Drive/Studies/KU Leuven/Courses/Classes/Y1S2/Advanced Analytics in Business/Project/3/AdvancedAnalytics_Streaming-Text-Analytics/"
folders_names = [file_path + 'data/' + folder for folder in os.listdir(file_path + 'data/') if "saved_stories" in folder]
compiled_stories = sc.textFile(','.join(folders_names[0:100]))

In [206]:
df = spark.read.json(compiled_stories)

In [124]:
df.show(5)

+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----+
|     aid|comments|              domain|frontpage|          posted_at|         source_text|        source_title|               title|                 url|          user|votes|
+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----+
|40273751|       0|         ervicee.com|    false|2024-05-06 12:18:16|ervicee - service...|ervicee - service...|Show HN: I made t...| https://ervicee.com| pedrofonsecaa|    1|
|40273924|       0|thezerostate.subs...|    false|2024-05-06 12:39:41|Naruto's Guide to...|Naruto's Guide to...|Naruto's Guide to...|https://thezerost...|         bcopa|    1|
|40273671|       0|         reuters.com|    false|2024-05-06 12:08:36|     reuters.com\n\n|         reuters.com|India EV

In [125]:
df.printSchema()

root
 |-- aid: string (nullable = true)
 |-- comments: long (nullable = true)
 |-- domain: string (nullable = true)
 |-- frontpage: boolean (nullable = true)
 |-- posted_at: string (nullable = true)
 |-- source_text: string (nullable = true)
 |-- source_title: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- user: string (nullable = true)
 |-- votes: long (nullable = true)



In [126]:
# Convert frontpage to numeric
df = df.withColumn('frontpage', when(df.frontpage==True, 1).otherwise(0))

In [127]:
# Check missing data and etc. 20% not front page
df.describe().toPandas()

,summary,aid,comments,domain,frontpage,posted_at,source_text,source_title,title,url,user,votes
0,count,307,307,307,307,307,307,300,307,307,307,307
1,mean,4.016383627035831E7,0.7882736156351792,None,0.1498371335504886,None,None,None,None,None,None,3.570032573289902
2,stddev,80842.22633604385,3.9029223946944507,None,0.35749442089287836,None,None,None,None,None,None,8.737127655708031
3,min,40050271,0,abc.net.au,0,2024-04-16 10:13:15,\n\n,'Architecture by conference' is a really bad idea,'Architecture by conference' is a bad idea,http://leehite.org/Chimes.htm,AlbeeDang,1
4,max,40274373,45,zork.net,1,2024-05-06 13:25:39,‘To the Future’: Saudi Arabia Spends Big to Be...,‘To the Future’: Saudi Arabia Spends Big to Be...,Zig 0.12.0 Release Notes,https://zork.net/~st/jottings/sais.html,zerojames,101


In [128]:
# Check out individual columns
#df.select('aid').show()
#df.select('comments').show()
#df.select('domain').show()
#df.select('frontpage').show()
#df.select('posted_at').show()
#df.select('source_text').show(n=1, truncate=False)
#df.select('source_title').show(n=5, truncate=False)
#df.select('title').show(n=5, truncate=False)
#df.select('url').show(n=5, truncate=False)
#df.select('title').show(n=5, truncate=False)
df.select('votes').show(n=5, truncate=False)

+-----+
|votes|
+-----+
|1    |
|1    |
|1    |
|2    |
|1    |
+-----+
only showing top 5 rows



In [129]:
# Checking for duplicates: Need to find a way to groupby
df.groupby('aid').agg({'aid':'count'}).orderBy(desc("count(aid)")).show()

+--------+----------+
|     aid|count(aid)|
+--------+----------+
|40273751|         1|
|40273924|         1|
|40273671|         1|
|40273894|         1|
|40273756|         1|
|40273772|         1|
|40273760|         1|
|40273913|         1|
|40273754|         1|
|40273683|         1|
|40273749|         1|
|40273936|         1|
|40273630|         1|
|40273619|         1|
|40273628|         1|
|40273629|         1|
|40273705|         1|
|40273711|         1|
|40273780|         1|
|40273892|         1|
+--------+----------+
only showing top 20 rows



In [130]:
# Extracting type of post: Show HN
def extract_ShowHN(title):
    if "Show HN" in title: 
        return "Y"
    else: 
        return "N"

extract_ShowHN_udf = udf(extract_ShowHN)

df = df.withColumn('isShowHN', extract_ShowHN_udf(df.title))
df.show(5)

+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----+-------+--------+
|     aid|comments|              domain|frontpage|          posted_at|         source_text|        source_title|               title|                 url|          user|votes|isAskHN|isShowHN|
+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------+-----+-------+--------+
|40273751|       0|         ervicee.com|        0|2024-05-06 12:18:16|ervicee - service...|ervicee - service...|Show HN: I made t...| https://ervicee.com| pedrofonsecaa|    1|      N|       Y|
|40273924|       0|thezerostate.subs...|        0|2024-05-06 12:39:41|Naruto's Guide to...|Naruto's Guide to...|Naruto's Guide to...|https://thezerost...|         bcopa|    1|      N|       N|
|40273671|       0|         reuters

In [131]:
# Extracting time of day
def extract_time_of_day(posted_at):
    return posted_at[11:13]

extract_time_of_day_udf = udf(extract_time_of_day)

df = df.withColumn('time_of_day', extract_time_of_day_udf(df.posted_at))

In [132]:
# Extracting day of week
from datetime import datetime

weekDay =  udf(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%w'))

df = df.withColumn('day_of_week', weekDay(df.posted_at))

In [186]:
# Train test split
train,test = df.randomSplit([0.8,0.2])


In [187]:
# Define numerical, categorical and target columns
TARGET_COL = ['frontpage']
NUM_COL = ['votes', 'comments']
CAT_COL = ['time_of_day', 'isShowHN', 'day_of_week']

In [188]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

# Creating numerical vector
numerical_vector_assembler = VectorAssembler(inputCols=NUM_COL,
                                             outputCol='num_col_vector')

train = numerical_vector_assembler.transform(train)
test = numerical_vector_assembler.transform(test)


In [189]:
# Standard scaling
scaler = StandardScaler(inputCol='num_col_vector',
                        outputCol='scaled_num_col_vector',
                        withStd=True, withMean=True)

scaler = scaler.fit(train)

train = scaler.transform(train)
test = scaler.transform(test)

In [190]:
df.printSchema()

root
 |-- aid: string (nullable = true)
 |-- comments: long (nullable = true)
 |-- domain: string (nullable = true)
 |-- frontpage: integer (nullable = false)
 |-- posted_at: string (nullable = true)
 |-- source_text: string (nullable = true)
 |-- source_title: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- user: string (nullable = true)
 |-- votes: long (nullable = true)
 |-- isAskHN: string (nullable = true)
 |-- isShowHN: string (nullable = true)
 |-- time_of_day: string (nullable = true)
 |-- day_of_week: string (nullable = true)



In [191]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='isShowHN',
                        outputCol='isShowHN_index')

indexer = indexer.fit(train)
train = indexer.transform(train)
test = indexer.transform(test)

train.show(3)

+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+-------+--------+-----------+-----------+--------------+---------------------+--------------+
|     aid|comments|              domain|frontpage|          posted_at|         source_text|        source_title|               title|                 url|         user|votes|isAskHN|isShowHN|time_of_day|day_of_week|num_col_vector|scaled_num_col_vector|isShowHN_index|
+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+-------+--------+-----------+-----------+--------------+---------------------+--------------+
|40273751|       0|         ervicee.com|        0|2024-05-06 12:18:16|ervicee - service...|ervicee - service...|Show HN: I made t...| https://ervicee.com|pedrofonsecaa|    1|      N|       Y|     

In [192]:
from pyspark.ml.feature import OneHotEncoder

one_hot_encoder = OneHotEncoder(inputCol='isShowHN_index',
                                outputCol='isShowHN_OHE')

one_hot_encoder = one_hot_encoder.fit(train)

train = one_hot_encoder.transform(train)
test = one_hot_encoder.transform(test)

train.show(3)

+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+-------+--------+-----------+-----------+--------------+---------------------+--------------+-------------+
|     aid|comments|              domain|frontpage|          posted_at|         source_text|        source_title|               title|                 url|         user|votes|isAskHN|isShowHN|time_of_day|day_of_week|num_col_vector|scaled_num_col_vector|isShowHN_index| isShowHN_OHE|
+--------+--------+--------------------+---------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----+-------+--------+-----------+-----------+--------------+---------------------+--------------+-------------+
|40273751|       0|         ervicee.com|        0|2024-05-06 12:18:16|ervicee - service...|ervicee - service...|Show HN: I made t...| https://ervicee.com|

In [193]:
# Compile s
assembler = VectorAssembler(inputCols=['scaled_num_col_vector',
                                       'isShowHN_OHE'],
                            outputCol='final_feature_vector')

train = assembler.transform(train)
test = assembler.transform(test)

In [194]:
train.select('final_feature_vector').take(5)

[Row(final_feature_vector=DenseVector([-0.2851, -0.199, 0.0])),
 Row(final_feature_vector=DenseVector([-0.2851, -0.199, 1.0])),
 Row(final_feature_vector=DenseVector([-0.2851, -0.199, 1.0])),
 Row(final_feature_vector=DenseVector([-0.1752, -0.199, 1.0])),
 Row(final_feature_vector=DenseVector([-0.2851, -0.199, 1.0]))]

In [195]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0, elasticNetParam=0.5, family="binomial",
                        featuresCol='final_feature_vector', labelCol='frontpage')

# Fit the model
lrModel = lr.fit(train)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))


Coefficients: [9.645817965512176,-0.41529264512651737,4.889823051624562]
Intercept: -6.102534133927195


In [196]:
# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = lrModel.summary

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))


+--------------------+-------------------+
|                 FPR|                TPR|
+--------------------+-------------------+
|                 0.0|                0.0|
|                 0.0|0.05128205128205128|
|                 0.0|0.07692307692307693|
|                 0.0|0.10256410256410256|
|                 0.0| 0.1282051282051282|
|                 0.0|0.15384615384615385|
|                 0.0| 0.1794871794871795|
|                 0.0|0.20512820512820512|
|                 0.0|0.23076923076923078|
|                 0.0| 0.2564102564102564|
|                 0.0|0.28205128205128205|
|                 0.0| 0.3076923076923077|
|                 0.0| 0.3333333333333333|
|0.004672897196261682|  0.358974358974359|
|0.004672897196261682|0.38461538461538464|
|0.004672897196261682|0.41025641025641024|
|0.004672897196261682| 0.4358974358974359|
|0.004672897196261682|0.46153846153846156|
|0.004672897196261682| 0.5128205128205128|
|0.009345794392523364| 0.5384615384615384|
+----------

In [197]:
# Set the model threshold to maximize F-Measure
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
lr.setThreshold(bestThreshold)

LogisticRegression_cc56f839b637

In [200]:
lr.getThreshold()

0.33220655140284583

In [201]:
# Gridsearch
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder().build()

# paramGrid = ParamGridBuilder() \
#     .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
#     .addGrid(lr.regParam, [0.1, 0.01]) \
#     .build()

crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=2)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(train)
print(cvModel.avgMetrics[0])



IllegalArgumentException: label does not exist. Available: aid, comments, domain, frontpage, posted_at, source_text, source_title, title, url, user, votes, isAskHN, isShowHN, time_of_day, day_of_week, num_col_vector, scaled_num_col_vector, isShowHN_index, isShowHN_OHE, final_feature_vector, CrossValidator_053f4b8aca4f_rand, rawPrediction, probability, prediction

In [ ]:
# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction = cvModel.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")


In [211]:
# Saving model
lrModel.save(file_path+"models/lrm_model")